In [1]:
from futpythontrader import *

In [2]:
key = "20e2d4aa7479a657c52935c4a743a626941b0a1f41b12156642dd7b62d893145"

In [3]:
# Realiza a requisição à API
url = "https://api.football-data-api.com/league-list?key=" + key + "&chosen_leagues_only=true"
data = requests.get(url)
resp = data.json()['data']

# Cria o DataFrame e aplica os filtros necessários
dados = pd.DataFrame.from_dict(resp)
dados = dados.sort_values(['name'])
dados = drop_reset_index(dados)

# Processa os dados em memória sem salvar e ler de arquivo
new_df = {'league': [], 'id': [], 'season': [], 'image': []}
for row in dados.itertuples():
    # Se row.season for string, converte com eval; caso contrário, utiliza diretamente
    season_data = eval(row.season) if isinstance(row.season, str) else row.season
    for i in season_data:
        new_df['league'].append(row.name)
        new_df['id'].append(i['id'])
        new_df['season'].append(i['year'])
        new_df['image'].append(row.image)

new = pd.DataFrame(new_df)
new.columns = ['League', 'ID', 'Season', 'Imagem']
new = drop_reset_index(new)

flt = (
    # (new.Season == 2021) | 
    # (new.Season == 2022) | 
    # (new.Season == 2023) | 
    (new.Season == 2024) | 
    (new.Season == 2025) | 
    # (new.Season == 20212022) | 
    # (new.Season == 20222023) | 
    (new.Season == 20232024) | 
    (new.Season == 20242025)  
    # | (new.Season == 20252026)
    )
new2 = new[flt]

ligas = drop_reset_index(new2)
display(ligas)

,League,ID,Season,Imagem
1,Algeria Ligue 1,10521,20232024,https://cdn.footystats.org/img/competitions/al...
2,Algeria Ligue 1,12665,20242025,https://cdn.footystats.org/img/competitions/al...
3,Argentina Prim B Nacional,11047,2024,https://cdn.footystats.org/img/competitions/ar...
4,Argentina Prim B Nacional,14008,2025,https://cdn.footystats.org/img/competitions/ar...
5,Argentina Primera División,11212,2024,https://cdn.footystats.org/img/competitions/ar...
...,...,...,...,...
255,Uzbekistan Uzbekistan Super League,14186,2025,https://cdn.footystats.org/img/competitions/uz...
256,Vietnam V.League 1,10721,20232024,https://cdn.footystats.org/img/competitions/vi...
257,Vietnam V.League 1,13152,20242025,https://cdn.footystats.org/img/competitions/vi...
258,Wales Welsh Premier League,9565,20232024,https://cdn.footystats.org/img/competitions/wa...


In [4]:
ligas['League'].unique()

array(['Algeria Ligue 1', 'Argentina Prim B Nacional',
       'Argentina Primera División', 'Australia A-League',
       'Australia New South Wales NPL', 'Austria 2. Liga',
       'Austria Bundesliga', 'Belarus Vysheyshaya Liga',
       'Belgium First Division B', 'Belgium Pro League',
       'Bosnia and Herzegovina Premier League of Bosnia',
       'Brazil Serie A', 'Brazil Serie B', 'Brazil Serie C',
       'Bulgaria First League', 'Canada Canadian Premier League',
       'Chile Primera B', 'Chile Primera División',
       'China China League One', 'China Chinese Super League',
       'Croatia Druga HNL', 'Croatia Prva HNL', 'Czech Republic FNL',
       'Czech Republic First League', 'Denmark Superliga',
       'Egypt Egyptian Premier League', 'England Championship',
       'England EFL League One', 'England EFL League Two',
       "England FA Women's Super League", 'England Premier League',
       'Estonia Meistriliiga', 'Europe UEFA Champions League',
       'FYR Macedonia First Fo

In [5]:
final_df = pd.DataFrame()

for i, j, k in zip(ligas.League, ligas.ID, ligas.Season):
    liga = str(i)
    season_id = str(j)
    year = str(k)
    
    try:
        url = f"https://api.football-data-api.com/league-matches?key={key}&season_id={season_id}"
        response = requests.get(url)
        resp_data = response.json().get('data', [])
        
        if resp_data:  
            df = pd.DataFrame.from_dict(resp_data)
            df['League'] = liga
            base_url = "https://footystats.org/img/"
            df['home_url'] = df['home_image'].apply(lambda x: base_url + x)
            df['away_url'] = df['away_image'].apply(lambda x: base_url + x)
            df['match_url'] = df['match_url'].apply(lambda x: base_url + x)
            # print(df.columns.to_list())
            
            final_df = pd.concat([final_df, df], ignore_index=True)
            
            print(f"Processado: {liga} - {year}")

    except Exception as e:
        print(f"Erro ao processar {liga} - {year}: {e}")
        continue

print("");print("")
print("Dados consolidados:")
display(final_df)


Processado: Algeria Ligue 1 - 20232024
Processado: Algeria Ligue 1 - 20242025
Processado: Argentina Prim B Nacional - 2024
Processado: Argentina Prim B Nacional - 2025
Processado: Argentina Primera División - 2024
Processado: Argentina Primera División - 2025
Processado: Australia A-League - 20232024
Processado: Australia A-League - 20242025
Processado: Australia New South Wales NPL - 2024
Processado: Australia New South Wales NPL - 2025
Processado: Austria 2. Liga - 20232024
Processado: Austria 2. Liga - 20242025
Processado: Austria Bundesliga - 20232024
Processado: Austria Bundesliga - 20242025
Processado: Belarus Vysheyshaya Liga - 2024
Processado: Belarus Vysheyshaya Liga - 2025
Processado: Belgium First Division B - 20232024
Processado: Belgium First Division B - 20242025
Processado: Belgium Pro League - 20232024
Processado: Belgium Pro League - 20242025
Processado: Bosnia and Herzegovina Premier League of Bosnia - 20232024
Processado: Bosnia and Herzegovina Premier League of Bosn

,id,homeID,awayID,season,status,roundID,game_week,revised_game_week,homeGoals,awayGoals,homeGoalCount,awayGoalCount,totalGoalCount,team_a_corners,team_b_corners,totalCornerCount,team_a_offsides,team_b_offsides,team_a_yellow_cards,team_b_yellow_cards,team_a_red_cards,team_b_red_cards,team_a_shotsOnTarget,team_b_shotsOnTarget,team_a_shotsOffTarget,team_b_shotsOffTarget,team_a_shots,team_b_shots,team_a_fouls,team_b_fouls,team_a_possession,team_b_possession,refereeID,coach_a_ID,coach_b_ID,stadium_name,stadium_location,team_a_cards_num,team_b_cards_num,odds_ft_1,odds_ft_x,odds_ft_2,odds_ft_over05,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_ft_under05,odds_ft_under15,odds_ft_under25,odds_ft_under35,odds_ft_under45,odds_btts_yes,odds_btts_no,odds_team_a_cs_yes,odds_team_a_cs_no,odds_team_b_cs_yes,odds_team_b_cs_no,odds_doublechance_1x,odds_doublechance_12,odds_doublechance_x2,odds_1st_half_result_1,odds_1st_half_result_x,odds_1st_half_result_2,odds_2nd_half_result_1,odds_2nd_half_result_x,odds_2nd_half_result_2,odds_dnb_1,odds_dnb_2,odds_corners_over_75,odds_corners_over_85,odds_corners_over_95,odds_corners_over_105,odds_corners_over_115,odds_corners_under_75,odds_corners_under_85,odds_corners_under_95,odds_corners_under_105,odds_corners_under_115,odds_corners_1,odds_corners_x,odds_corners_2,odds_team_to_score_first_1,odds_team_to_score_first_x,odds_team_to_score_first_2,odds_win_to_nil_1,odds_win_to_nil_2,odds_1st_half_over05,odds_1st_half_over15,odds_1st_half_over25,odds_1st_half_over35,odds_1st_half_under05,odds_1st_half_under15,odds_1st_half_under25,odds_1st_half_under35,odds_2nd_half_over05,odds_2nd_half_over15,odds_2nd_half_over25,odds_2nd_half_over35,odds_2nd_half_under05,odds_2nd_half_under15,odds_2nd_half_under25,odds_2nd_half_under35,odds_btts_1st_half_yes,odds_btts_1st_half_no,odds_btts_2nd_half_yes,odds_btts_2nd_half_no,overallGoalCount,ht_goals_team_a,ht_goals_team_b,goals_2hg_team_a,goals_2hg_team_b,GoalCount_2hg,HTGoalCount,date_unix,winningTeam,no_home_away,btts_potential,btts_fhg_potential,btts_2hg_potential,goalTimingDisabled,attendance,corner_timings_recorded,card_timings_recorded,team_a_fh_corners,team_b_fh_corners,team_a_2h_corners,team_b_2h_corners,corner_fh_count,corner_2h_count,team_a_fh_cards,team_b_fh_cards,team_a_2h_cards,team_b_2h_cards,total_fh_cards,total_2h_cards,attacks_recorded,team_a_dangerous_attacks,team_b_dangerous_attacks,team_a_attacks,team_b_attacks,team_a_xg,team_b_xg,total_xg,team_a_penalties_won,team_b_penalties_won,team_a_penalty_goals,team_b_penalty_goals,team_a_penalty_missed,team_b_penalty_missed,pens_recorded,goal_timings_recorded,team_a_0_10_min_goals,team_b_0_10_min_goals,team_a_corners_0_10_min,team_b_corners_0_10_min,team_a_cards_0_10_min,team_b_cards_0_10_min,throwins_recorded,team_a_throwins,team_b_throwins,freekicks_recorded,team_a_freekicks,team_b_freekicks,goalkicks_recorded,team_a_goalkicks,team_b_goalkicks,o45_potential,o35_potential,o25_potential,o15_potential,o05_potential,o15HT_potential,o05HT_potential,o05_2H_potential,o15_2H_potential,corners_potential,offsides_potential,cards_potential,avg_potential,home_url,home_image,home_name,away_url,away_image,away_name,home_ppg,away_ppg,pre_match_home_ppg,pre_match_away_ppg,pre_match_teamA_overall_ppg,pre_match_teamB_overall_ppg,u45_potential,u35_potential,u25_potential,u15_potential,u05_potential,corners_o85_potential,corners_o95_potential,corners_o105_potential,team_a_xg_prematch,team_b_xg_prematch,total_xg_prematch,match_url,competition_id,matches_completed_minimum,over05,over15,over25,over35,over45,over55,btts,homeGoals_timings,awayGoals_timings,League
0,7190999,666696,1291,2023/2024,complete,103405,1,-1,[61],[],1,0,1,9,2,11,-1,-1,2,2,0,0,8,2,7,2,15,4,-1,-1,58,42,NaN,2898.0,2936.0,,,2,2,1.60,3.40,5.00,1.07,1.50,2.50,4.50,9.3,6.45,2.45,1.50,1.18,1.01,2.63,1.44,2.10,1.67,3.40,1.29,1.20,1.35,1.82,2.30,1.95,6.50,2.05,2.20,5.50,0,0,1.69,2.17,2.85,4.1,6.00,2.11,1.66,1.37,1.19,1.09,1.45,8.30,3.25,1.44,6.25,3.75,0

In [14]:
final_df.to_csv("Base_de_Dados_FootyStats_Full.csv", index=False)

In [6]:
df = final_df[['id','status','date_unix','League','season','game_week','home_name','away_name',
'homeGoalCount','awayGoalCount','totalGoalCount',
'homeGoals','awayGoals',
'odds_ft_1','odds_ft_x','odds_ft_2',
'odds_doublechance_1x','odds_doublechance_12','odds_doublechance_x2',
'odds_ft_over05','odds_ft_over15','odds_ft_over25','odds_ft_over35','odds_ft_over45',
'odds_ft_under05','odds_ft_under15','odds_ft_under25','odds_ft_under35','odds_ft_under45',
'odds_btts_yes','odds_btts_no',
'ht_goals_team_a','ht_goals_team_b','HTGoalCount',
'odds_1st_half_result_1','odds_1st_half_result_x','odds_1st_half_result_2',
'odds_1st_half_over05','odds_1st_half_over15','odds_1st_half_over25',
'odds_1st_half_under05','odds_1st_half_under15','odds_1st_half_under25',
'team_a_xg_prematch','team_b_xg_prematch','total_xg_prematch',
'team_a_xg','team_b_xg','total_xg',
'team_a_possession','team_b_possession',
'team_a_attacks','team_b_attacks',
'team_a_dangerous_attacks','team_b_dangerous_attacks',
'team_a_shots','team_b_shots','team_a_shotsOnTarget','team_b_shotsOnTarget','team_a_shotsOffTarget','team_b_shotsOffTarget',
'match_url','home_url','away_url']]

df.columns = ['id','status','date','league','season','round','home','away',
'goals_h_ft','goals_a_ft','total_goals_ft',
'goals_h_minutes','goals_a_minutes',
'odd_h_ft','odd_d_ft','odd_a_ft',
'odd_1x','odd_12','odd_x2',
'odd_over05_ft','odd_over15_ft','odd_over25_ft','odd_over35_ft','odd_over45_ft',
'odd_under05_ft','odd_under15_ft','odd_under25_ft','odd_under35_ft','odd_under45_ft',
'odds_btts_yes','odds_btts_no',
'goals_h_ht','goals_a_ht','total_goals_ht',
'odd_h_ht','odd_d_ht','odd_a_ht',
'odd_over05_ht','odd_over15_ht','odd_over25_ht',
'odd_under05_ht','odd_under15_ht','odd_under25_ht',
'xg_h_prematch','xg_a_prematch','xg_total_prematch',
'xg_h','xg_a','xg_total',
'possession_h','possession_a',
'attacks_h','attacks_a',
'dangerous_attacks_h','dangerous_attacks_a',
'shots_h','shots_a',
'shots_on_target_h','shots_on_target_a',
'shots_off_target_h','shots_off_target_a',
'match_url','home_url','away_url']

df.fillna(0, inplace=True)
df = drop_reset_index(df)

display(df)
df.to_csv("Base_de_Dados_FootyStats.csv", index=False)

,id,status,date,league,season,round,home,away,goals_h_ft,goals_a_ft,total_goals_ft,goals_h_minutes,goals_a_minutes,odd_h_ft,odd_d_ft,odd_a_ft,odd_1x,odd_12,odd_x2,odd_over05_ft,odd_over15_ft,odd_over25_ft,odd_over35_ft,odd_over45_ft,odd_under05_ft,odd_under15_ft,odd_under25_ft,odd_under35_ft,odd_under45_ft,odds_btts_yes,odds_btts_no,goals_h_ht,goals_a_ht,total_goals_ht,odd_h_ht,odd_d_ht,odd_a_ht,odd_over05_ht,odd_over15_ht,odd_over25_ht,odd_under05_ht,odd_under15_ht,odd_under25_ht,xg_h_prematch,xg_a_prematch,xg_total_prematch,xg_h,xg_a,xg_total,possession_h,possession_a,attacks_h,attacks_a,dangerous_attacks_h,dangerous_attacks_a,shots_h,shots_a,shots_on_target_h,shots_on_target_a,shots_off_target_h,shots_off_target_a,match_url,home_url,away_url
1,7190999,complete,1694790000,Algeria Ligue 1,2023/2024,1,USM Khenchela,ES Sétif,1,0,1,[61],[],1.60,3.40,5.00,1.20,1.35,1.82,1.07,1.50,2.50,4.50,9.3,6.45,2.45,1.50,1.18,1.01,2.63,1.44,0,0,0,2.30,1.95,6.50,1.55,3.40,9.75,2.30,1.27,1.05,0.00,0.00,0.00,1.75,0.51,2.27,58,42,64,45,35,16,15,4,8,2,7,2,https://footystats.org/img//algeria/es-setif-v...,https://footystats.org/img/teams/algeria-usm-k...,https://footystats.org/img/teams/algeria-es-se...
2,7191000,complete,1695826800,Algeria Ligue 1,2023/2024,1,Paradou AC,USM Alger,1,0,1,[90+3],[],2.50,2.90,2.70,1.17,1.28,2.05,1.05,1.44,2.63,4.50,8.4,7.10,2.65,1.44,1.18,1.03,2.20,1.62,0,0,0,3.25,1.91,3.60,1.50,3.40,9.75,2.40,1.27,1.05,0.00,0.00,0.00,1.59,1.58,3.17,53,47,126,118,82,92,9,10,6,4,3,6,https://footystats.org/img//algeria/paradou-ac...,https://footystats.org/img/teams/algeria-parad...,https://footystats.org/img/teams/algeria-usm-a...
3,7191001,complete,1694876400,Algeria Ligue 1,2023/2024,1,NC Magra,JS Kabylie,0,1,1,[],[44],3.30,2.88,2.15,1.58,1.37,1.29,1.13,1.63,3.10,5.80,11.5,5.50,2.16,1.36,1.09,1.03,2.50,1.50,0,1,1,4.33,1.80,3.00,1.63,3.90,12.00,2.10,1.22,1.03,0.00,0.00,0.00,1.46,1.40,2.86,62,38,150,116,82,56,9,9,4,6,5,3,https://footystats.org/img//algeria/js-de-kaby...,https://footystats.org/img/teams/algeria-nc-ma...,https://footystats.org/img/teams/algeria-js-de...
4,7191002,complete,1697207400,Algeria Ligue 1,2023/2024,1,US Souf,CR Belouizdad,0,3,3,[],"[16, 43, 45+1]",3.90,3.10,1.95,1.67,1.29,1.25,1.05,1.42,2.38,4.50,8.4,7.10,2.70,1.53,1.18,1.03,2.20,1.62,0,3,3,4.75,1.95,2.63,1.50,3.40,7.80,2.40,1.29,1.05,1.43,0.00,1.43,0.00,0.00,0.00,-1,-1,0,0,0,0,-1,-1,-1,-1,-1,-1,https://footystats.org/img//algeria/cr-belouiz...,https://footystats.org/img/teams/algeria-us-so...,https://footystats.org/img/teams/algeria-cr-be...
5,7191003,complete,1694800800,Algeria Ligue 1,2023/2024,1,ASO Chlef,MC Oran,2,0,2,"[47, 62]",[],1.62,3.25,5.75,1.13,1.29,2.20,1.09,1.45,2.55,4.50,9.0,6.75,2.63,1.47,1.18,1.06,2.25,1.57,0,0,0,2.40,1.95,5.50,1.51,3.30,9.50,2.35,1.28,1.05,0.00,0.00,0.00,1.58,0.85,2.43,61,39,77,55,58,27,14,7,4,3,10,4,https://footystats.org/img//algeria/mc-oran-vs...,https://footystats.org/img/teams/algeria-aso-c...,https://footystats.org/img/teams/algeria-mc-or...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69192,7817228,complete,1745063100,Wales Welsh Premier League,2024/2025,10,Flint Town United,Newtown,4,0,4,"[27, 31, 43, 83]",[],2.16,3.51,3.11,1.27,1.22,1.77,1.01,1.10,1.53,2.25,3.9,13.00,5.20,2.35,1.57,1.18,1.53,2.38,3,0,3,2.40,2.38,3.60,1.29,2.25,5.00,3.50,1.57,1.14,1.84,1.40,3.24,1.99,1.63,3.62,50,50,85,92,86,72,13,12,8,5,5,7,https://footystats.org/img//wales/newtown-afc-...,https://footystats.org/img/teams/wales-flint-t...,https://footystats.org/img/teams/wales-newtown...
69193,7885739,complete,1745674200,Wales Welsh Premier League,2024/2025,0,Cardiff MU,Bala Town,1,0,1,[90+3],[],2.20,3.30,3.20,0.00,0.00,0.00,0.00,0.00,1.92,0.00,0.0,0.00,0.00,1.82,0.00,0.00,1.73,2.00,0,0,0,2.88,2.10,3.60,1.40,3.00,7.00,2.75,1.36,1.08,1.32,1.2

In [7]:
# league_replacements_footystats = {
#     "Algeria Ligue 1": "ALGERIA 1",
#     "Argentina Primera División": "ARGENTINA 1",
#     "Australia A-League": "AUSTRALIA 1",
#     "Australia New South Wales NPL": "AUSTRALIA 2",
#     "Austria Bundesliga": "AUSTRIA 1",
#     "Austria 2. Liga": "AUSTRIA 2",
#     "Belarus Vysheyshaya Liga": "BELARUS 1",
#     "Belgium Pro League": "BELGIUM 1",
#     "Belgium First Division B": "BELGIUM 2",
#     "Bosnia and Herzegovina Premier League of Bosnia": "BOSNIA 1",
#     "Brazil Serie A": "BRAZIL 1",
#     "Brazil Serie B": "BRAZIL 2",
#     "Brazil Serie C": "BRAZIL 3",
#     "Bulgaria First League": "BULGARIA 1",
#     "Canada Canadian Premier League": "CANADA 1",
#     "Chile Primera División": "CHILE 1",
#     "Chile Primera B": "CHILE 2",
#     "China Chinese Super League": "CHINA 1",
#     "China China League One": "CHINA 2",
#     "South America Copa Libertadores": "COPA LIBERTADORES",
#     "Croatia Prva HNL": "CROATIA 1",
#     "Croatia Druga HNL": "CROATIA 2",
#     "Czech Republic First League": "CZECH 1",
#     "Czech Republic FNL": "CZECH 2",
#     "Denmark Superliga": "DENMARK 1",
#     "Egypt Egyptian Premier League": "EGYPT 1",
#     "England Premier League": "ENGLAND 1",
#     "England Championship": "ENGLAND 2",
#     "England EFL League One": "ENGLAND 3",
#     "England EFL League Two": "ENGLAND 4",
#     "England FA Women's Super League": "WOMEN ENGLAND 1",
#     "Estonia Meistriliiga": "ESTONIA 1",
#     "Europe UEFA Champions League": "EUROPA CHAMPIONS LEAGUE",
#     "Faroe Islands Faroe Islands Premier League": "FAROE ISLANDS 1",
#     "Finland Veikkausliiga": "FINLAND 1",
#     "France Ligue 1": "FRANCE 1",
#     "France Ligue 2": "FRANCE 2",
#     "France National": "FRANCE 3",
#     "FYR Macedonia First Football League": "MACEDONIA 1",
#     "Georgia Erovnuli Liga": "GEORGIA 1",
#     "Germany Bundesliga": "GERMANY 1",
#     "Germany 2. Bundesliga": "GERMANY 2",
#     "Germany 3. Liga": "GERMANY 3",
#     "Greece Super League": "GREECE 1",
#     "Hungary NB I": "HUNGARY 1",
#     "Hungary NB II": "HUNGARY 2",
#     "Iceland Úrvalsdeild": "ICELAND 1",
#     "Indonesia Liga 1": "INDONESIA 1",
#     "Iran Persian Gulf Pro League": "IRAN 1",
#     "Ireland Premier Division": "IRELAND 1",
#     "Republic of Ireland Premier Division": "IRELAND 1",
#     "Republic of Ireland First Division": "IRELAND 2",
#     "Israel Israeli Premier League": "ISRAEL 1",
#     "Israel Liga Leumit": "ISRAEL 2",
#     "Italy Serie A": "ITALY 1",
#     "Italy Serie B": "ITALY 2",
#     "Italy Serie C": "ITALY 3",
#     "Italy Serie C Group A": "ITALY 3",
#     "Italy Serie C Group B": "ITALY 3",
#     "Italy Serie C Group C": "ITALY 3",
#     "Jamaica Jamaica National Premier League": "JAMAICA 1",
#     "Japan J1 League": "JAPAN 1",
#     "Japan J2 League": "JAPAN 2",
#     "Jordan Jordanian Pro League": "JORDAN 1",
#     "Kazakhstan Kazakhstan Premier League": "KAZAKHSTAN 1",
#     "Luxembourg National Division": "LUXEMBOURG 1",
#     "Malaysia Super League": "MALAYSIA 1",
#     "Montenegro Montenegrin First League": "MONTENEGRO 1",
#     "Morocco Botola Pro": "MOROCCO 1",
#     "Netherlands Eredivisie": "NETHERLANDS 1",
#     "Netherlands Eerste Divisie": "NETHERLANDS 2",
#     "Netherlands Eredivisie Women": "WOMEN NETHERLANDS 1",
#     "Nigeria NPFL": "NIGERIA 1",
#     "Norway Eliteserien": "NORWAY 1",
#     "Norway First Division": "NORWAY 2",
#     "Norway Toppserien": "WOMEN NORWAY 1",
#     "Northern Ireland NIFL Premiership": "NORTHERN IRELAND 1",
#     "Oman Professional League": "OMAN 1",
#     "Paraguay Division Profesional": "PARAGUAY 1",
#     "Poland Ekstraklasa": "POLAND 1",
#     "Poland 1. Liga": "POLAND 2",
#     "Portugal Liga NOS": "PORTUGAL 1",
#     "Portugal LigaPro": "PORTUGAL 2",
#     "Portugal Liga 3": "PORTUGAL 3",
#     "Portugal Campeonato de Portugal Group A": "PORTUGAL 4",
#     "Portugal Campeonato de Portugal Group B": "PORTUGAL 4",
#     "Portugal Campeonato de Portugal Group C": "PORTUGAL 4",
#     "Portugal Campeonato de Portugal Group D": "PORTUGAL 4",
#     "Qatar Stars League": "QATAR 1",
#     "Romania Liga I": "ROMANIA 1",
#     "Russia Russian Premier League": "RUSSIA 1",
#     "Russia FNL": "RUSSIA 2",
#     "Saudi Arabia Professional League": "SAUDI ARABIA 1",
#     "Scotland Premiership": "SCOTLAND 1",
#     "Scotland Championship": "SCOTLAND 2",
#     "Serbia SuperLiga": "SERBIA 1",
#     "Serbia Prva Liga": "SERBIA 2",
#     "Singapore S.League": "SINGAPORE 1",
#     "Slovakia Super Liga": "SLOVAKIA 1",
#     "Slovenia PrvaLiga": "SLOVENIA 1",
#     "South Africa Premier Soccer League": "SOUTH AFRICA 1",
#     "South Korea K League 1": "SOUTH KOREA 1",
#     "South Korea K League 2": "SOUTH KOREA 2",
#     "Spain La Liga": "SPAIN 1",
#     "Spain Segunda División": "SPAIN 2",
#     "Spain Primera Division RFEF Group 1": "SPAIN 3",
#     "Spain Primera Division RFEF Group 2": "SPAIN 3",
#     "Spain Segunda División RFEF Group 1": "SPAIN 4",
#     "Spain Segunda División RFEF Group 2": "SPAIN 4",
#     "Spain Segunda División RFEF Group 3": "SPAIN 4",
#     "Spain Segunda División RFEF Group 4": "SPAIN 4",
#     "Spain Segunda División RFEF Group 5": "SPAIN 4",
#     "Spain Primera Division Women": "WOMEN SPAIN 1",
#     "Sweden Allsvenskan": "SWEDEN 1",
#     "Sweden Superettan": "SWEDEN 2",
#     "Sweden Damallsvenskan": "WOMEN SWEDEN 1",
#     "Switzerland Super League": "SWITZERLAND 1",
#     "Switzerland Challenge League": "SWITZERLAND 2",
#     "Taiwan Taiwan Football Premier League": "TAIWAN 1",
#     "Thailand Thai League T1": "THAILAND 1",
#     "Tunisia Ligue 1": "TUNISIA 1",
#     "Turkey Süper Lig": "TURKEY 1",
#     "Turkey 1. Lig": "TURKEY 2",
#     "UAE Arabian Gulf League": "UAE 1",
#     "Ukraine Ukrainian Premier League": "UKRAINE 1",
#     "Uruguay Primera División": "URUGUAY 1",
#     "USA MLS": "USA 1",
#     "USA USL Championship": "USA 2",
#     "USA MLS Next Pro": "USA 3",
#     "USA NWSL": "WOMEN USA 1",
#     "Uzbekistan Uzbekistan Super League": "UZBEKISTAN 1",
#     "Vietnam V.League 1": "VIETNAM 1",
#     "Wales Welsh Premier League": "WALES 1",
# }


# def replace_league_name(league_name):
#     return league_replacements_footystats.get(league_name, league_name)

# df['league'] = df['league'].apply(replace_league_name)

# leagues = df.sort_values(by='league')
# leagues = leagues['league'].unique()
# print(len(leagues))
# leagues

In [8]:
# leagues_footystats = [ 
# 'ARGENTINA 1', 
# 'AUSTRALIA 1',
# 'AUSTRIA 1', 'AUSTRIA 2', 
# 'BELGIUM 1', 'BELGIUM 2', 
# 'BOSNIA 1',
# 'BRAZIL 1', 'BRAZIL 2', 'BRAZIL 3', 
# 'BULGARIA 1', 
# 'CHILE 1', 'CHILE 2', 
# 'CHINA 1', 'CHINA 2', 
# 'COPA LIBERTADORES',
# 'CROATIA 1', 'CROATIA 2', 
# 'CZECH 1', 'CZECH 2', 
# 'DENMARK 1',
# 'EGYPT 1', 
# 'ENGLAND 1', 'ENGLAND 2', 'ENGLAND 3', 'ENGLAND 4',
# 'ESTONIA 1', 
# 'EUROPA CHAMPIONS LEAGUE', 
# 'FINLAND 1', 
# 'FRANCE 1', 'FRANCE 2', 'FRANCE 3', 
# 'GERMANY 1', 'GERMANY 2', 'GERMANY 3', 
# 'GREECE 1', 
# 'HUNGARY 1', 'HUNGARY 2', 
# 'ICELAND 1', 
# 'IRELAND 1', 'IRELAND 2', 
# 'ISRAEL 1', 'ISRAEL 2', 
# 'ITALY 1', 'ITALY 2', 'ITALY 3', 
# 'JAPAN 1', 'JAPAN 2', 
# 'NETHERLANDS 1', 'NETHERLANDS 2',
# 'NORTHERN IRELAND 1', 
# 'NORWAY 1', 'NORWAY 2',
# 'PARAGUAY 1', 
# 'POLAND 1', 'POLAND 2', 
# 'PORTUGAL 1', 'PORTUGAL 2', 'PORTUGAL 3',
# 'ROMANIA 1',
# 'SAUDI ARABIA 1', 
# 'SCOTLAND 1', 'SCOTLAND 2', 
# 'SERBIA 1', 'SERBIA 2', 
# 'SLOVAKIA 1',
# 'SLOVENIA 1', 
# 'SOUTH AFRICA 1', 
# 'SOUTH KOREA 1', 'SOUTH KOREA 2',
# 'SPAIN 1', 'SPAIN 2', 'SPAIN 3', 
# 'SWEDEN 1', 'SWEDEN 2',
# 'SWITZERLAND 1', 'SWITZERLAND 2', 
# 'TURKEY 1', 'TURKEY 2', 
# 'UKRAINE 1', 
# 'USA 1',
# 'WALES 1']

# print(len(leagues_footystats))

In [9]:
# # df = df[df['league'].isin(leagues_footystats)]
# df['date'] = pd.to_datetime(df['date'], unit='s') - pd.DateOffset(hours=3)
# df = df.sort_values(['date'])
# df['time'] = df['date'].dt.time
# df['date'] = df['date'].dt.date
# df = drop_reset_index(df)
# df = df[['id','status','date','time','league','season','round','home','away',
# 'goals_h_ft','goals_a_ft','total_goals_ft',
# 'goals_h_minutes','goals_a_minutes',
# 'odd_h_ft','odd_d_ft','odd_a_ft',
# 'odd_1x','odd_12','odd_x2',
# 'odd_over05_ft','odd_over15_ft','odd_over25_ft','odd_over35_ft','odd_over45_ft',
# 'odd_under05_ft','odd_under15_ft','odd_under25_ft','odd_under35_ft','odd_under45_ft',
# 'odds_btts_yes','odds_btts_no',
# 'goals_h_ht','goals_a_ht','total_goals_ht',
# 'odd_h_ht','odd_d_ht','odd_a_ht',
# 'odd_over05_ht','odd_over15_ht','odd_over25_ht',
# 'odd_under05_ht','odd_under15_ht','odd_under25_ht',
# 'xg_h_prematch','xg_a_prematch','xg_total_prematch',
# 'xg_h','xg_a','xg_total',
# 'possession_h','possession_a',
# 'attacks_h','attacks_a',
# 'dangerous_attacks_h','dangerous_attacks_a',
# 'shots_h','shots_a',
# 'shots_on_target_h','shots_on_target_a',
# 'shots_off_target_h','shots_off_target_a',
# 'match_url','home_url','away_url']]
# display(df)

In [10]:
# df['date'] = pd.to_datetime(df['date']).dt.date

# n_per = 5

# df['ptos_h'] = np.where(df['goals_h_ft'] > df['goals_a_ft'], 3,
#                 np.where(df['goals_h_ft'] == df['goals_a_ft'], 1, 0))

# df['ptos_a'] = np.where(df['goals_h_ft'] < df['goals_a_ft'], 3,
#                 np.where(df['goals_h_ft'] == df['goals_a_ft'], 1, 0))

# df['media_ptos_h'] = df.groupby('home')['ptos_h'].shift(1).rolling(window=n_per, min_periods=1).mean().reset_index(0, drop=True)
# df['media_ptos_a'] = df.groupby('away')['ptos_a'].shift(1).rolling(window=n_per, min_periods=1).mean().reset_index(0, drop=True)

# df.fillna(0, inplace=True)
# df = drop_reset_index(df)
# df

In [11]:
# rename_teams(df)
# df

In [12]:
# df.to_parquet("base_footystats_four_traders_and_one_punter.parquet", index=False)